In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py
from tqdm import tqdm
import os


from holodeck import plot, detstats, utils
import holodeck.single_sources as sings
from holodeck.constants import YR, MSOL, MPC, GYR, PC
import holodeck as holo
from holodeck.sams import sam

import hasasia.sim as hsim

In [ ]:
target = 'hard_time'
data, params, dsdat = detstats.get_data(target,
        gw_only=False, red2white=None, red_gamma=None,
        nloudest=10, bgl=1, nexcl=0,
        )


In [ ]:
PP = 10
dp_ss = dsdat[PP]['dp_ss']
fobs = data[PP]['fobs_cents']

In [ ]:
print(dp_ss.shape)
print(np.unravel_index(np.argmax(dp_ss), shape=dp_ss.shape))

### find a realization that can have high dp_ss

In [ ]:
RR, ss = np.unravel_index(np.argmax(dp_ss), shape=dp_ss.shape)
print(f"{RR=}, {ss=}, {dp_ss[RR,ss]=}")

check the individual gamma_ssi

In [ ]:
gamma_ssi = dsdat[PP]['gamma_ssi'][:,RR,ss]
ev_ss = dsdat[PP]['ev_ss'][RR]
print(gamma_ssi.shape)
fig, ax = plot.figax(xlabel='fobs', ylabel='DP_i')
ax.scatter(np.repeat(fobs, gamma_ssi[0].size).reshape(gamma_ssi.shape), gamma_ssi)


check other sky realizations, many have high dp_ss. nice

In [ ]:
print(np.sum(dp_ss[RR]))
fig, ax = plot.figax(xlabel='sky realizations', ylabel='dp_ss', xscale='linear')
ax.scatter(np.arange(len(dp_ss[RR])), dp_ss[RR], alpha=0.5)
fig
fig, ax = plot.figax(xlabel='sky realizations', ylabel='ev_ss', xscale='linear')
ax.scatter(np.arange(len(ev_ss)), ev_ss, alpha=0.5, color='orange')

use the hc_ss and hc_bg from this realization

In [ ]:
fig, ax = plot.figax(xlabel='fobs', ylabel='hc')
hc_bg, hc_ss = data[PP]['hc_bg'][:,RR], data[PP]['hc_ss'][:,RR]
ax.plot(fobs, hc_bg)
ax.scatter(np.repeat(fobs, hc_ss[0].size).reshape(hc_ss.shape), hc_ss,
           alpha=0.25)

# make a pta for fresh detstats

In [ ]:

def check_detstats(seed, hc_ss, hc_bg):
    # seed = None
    if seed is None:
        seed = np.random.randint(99999) # get a random number
    # print(f"{seed=}")
    np.random.seed(seed)


    # thetas = None
    thetas = np.repeat(np.pi/2, 40)
    psrs, red_amp, sigma, _, _ = detstats.calibrate_one_pta(hc_bg, hc_ss, fobs, npsrs=40, seed=seed,
                                                            ret_sig=True, thetas=thetas)

    # print(f"{red_amp=}, {sigma=}")

    # check distinction/matching w random seed
    max_phi, max_theta = 0, 0
    am_phi, am_theta = None, None
    for ii, psr in enumerate(psrs):
        if psr.phi > max_phi:
            max_phi = psr.phi
            am_phi = ii
        if psr.theta > max_theta:
            max_theta = psr.theta
            am_theta = ii
        

    _dp_bg, _snr_bg = detstats.detect_bg_pta(psrs, fobs, hc_bg[:,np.newaxis], ret_snr=True)
    print(f"{_dp_bg[0]=:.2f}, {_snr_bg[0]=:.2f}, {am_theta=}, {am_phi=}")

    # get single source detstats
    _dp_ss, _snr_ss, _gamma_ssi = detstats.detect_ss_pta(
            psrs, fobs, hc_ss[:,np.newaxis], hc_bg[:,np.newaxis], 
            nskies=1, ret_snr=True,)
    # print(f"sum(DP_ssi)={np.sum(_gamma_ssi)}")
    print(f"<N_SS> ={detstats.expval_of_ss(_gamma_ssi)[0,0]:.3f}, N_SS = {detstats.count_n_ss(_gamma_ssi)[0,0]}")

hc_bg, hc_ss = data[PP]['hc_bg'][:,RR], data[PP]['hc_ss'][:,RR]
check_detstats(seed=None, hc_ss=hc_ss, hc_bg=hc_bg)

# Artificially inject a second equally loud source

In [ ]:
hc_ss2 = np.copy(hc_ss)
hc_ss2[0,1] = hc_ss2[0,0]
fig, ax = plot.figax(xlabel='fobs', ylabel='hc')
ax.plot(fobs, hc_bg)
ax.scatter(np.repeat(fobs, hc_ss2[0].size).reshape(hc_ss.shape), hc_ss,
           alpha=0.25)



### check detstats for 2 equally loud

In [ ]:
for i in range(10):
    check_detstats(seed=None, hc_ss=hc_ss2, hc_bg=hc_bg)

# Conclusion 1
When I have 1 detectable, very loud source, and I make the second source equally loud, i get two detections!

In [ ]:
hc_ss3 = np.copy(hc_ss)
hc_ss3[0,0] = 6*10**-15
hc_ss3[0,1] = 6*10**-15
fig, ax = plot.figax(xlabel='fobs', ylabel='hc')
ax.plot(fobs, hc_bg)
ax.scatter(np.repeat(fobs, hc_ss3[0].size).reshape(hc_ss3.shape), hc_ss3,
           alpha=0.25)
for ii in range(10):
    check_detstats(seed=None, hc_ss=hc_ss3, hc_bg=hc_bg)

# Try different arbritrarily assigned loudest

# Check for a pspace library how often we get two very loud sources, like the reproduced ones here

In [ ]:
print(f"{params[0]['hard_time']}")

In [ ]:

fig, ax = plot.figax(xlabel='1st hc_ss', ylabel='2nd hc_ss',
                    figsize=(6,6))
ax.axvline(4e-15, color='k', ls='--', alpha=0.25)
ax.axhline(4e-15, color='k', ls='--', alpha=0.25)
handles = []
for pp in np.arange(0,21,5):
    print(pp)
    ssall = data[pp]['hc_ss']
    label=f"$\\tau_f$={params[pp]['hard_time']}"
    hh = ax.scatter(ssall[0,:,0], ssall[0,:,1], alpha=0.5, label=label)
    handles.append(hh)
ax.legend(handles=handles)

In [ ]:
for pp in np.arange(0,21,5):
    print(pp)
    ssall = data[pp]['hc_ss']
    fig, ax = plot.figax(xlabel='1st hc_ss', ylabel='2nd hc_ss',
                        figsize=(5,5))
    ax.scatter(ssall[0,:,0], ssall[0,:,1], alpha=0.5)
    ax.axvline(4e-15, color='k', ls='--', alpha=0.25)
    ax.axhline(4e-15, color='k', ls='--', alpha=0.25)
    ax.set_title(f"$\\tau_f$={params[pp]['hard_time']}")